In [1]:
import pandas as pd, numpy as np
from drdid import drdid, reg_did

from att_gt.compute_attgt import panel2cs2
from att_gt.pre_process_did import pre_process_did
from att_gt.compute_attgt import compute_att_gt

import statsmodels.api as sm
import patsy 
fml = patsy.dmatrices

lm = sm.WLS
n_x = np.newaxis
qr_solver = np.linalg.pinv



In [2]:
data = pd.read_csv("../data/sim_data.csv")
data.head()

,Unnamed: 0,G,X,id,period,Y
0,1,4,-0.876233,1,1,3.248701
1,2,4,-0.876233,1,2,2.266837
2,3,4,-0.876233,1,3,3.990885
3,4,4,-0.876233,1,4,4.653489
4,5,4,0.974974,2,1,5.880457


In [5]:
yname = "Y"
tname = "period"
idname = "id"
gname = "G"
data = data
xformla = "Y~X"
dp = pre_process_did(idname=idname, yname=yname, tname=tname, gname=gname, data = data, xformla= xformla)

dp.keys()

op, inf = compute_att_gt(dp)
op

{'group': [2, 2, 2, 3, 3, 3, 4, 4, 4],
 'year': [2, 3, 4, 2, 3, 4, 2, 3, 4],
 'att': [0.8363102379527425,
  2.181344332350583,
  3.7464899277642427,
  -0.6523370074510487,
  1.6078312816887652,
  3.098371797652095,
  -0.10768167926179448,
  0.2512301135435509,
  1.439661425578744],
 'post ': [1, 1, 1, 0, 1, 1, 0, 0, 1]}

In [6]:
dp["biters"] = 1000
dp['alp'] = 0.05
dp['true_repeated_cross_sections'] = dp['true_rep_cross_section']
didparams = dp
dp.keys()

dict_keys(['yname', 'tname', 'idname', 'gname', 'xformla', 'data', 'tlist', 'glist', 'n', 'nG', 'nT', 'control_group', 'anticipation', 'weights_name', 'panel', 'true_rep_cross_section', 'clustervars', 'biters', 'alp', 'true_repeated_cross_sections'])

In [7]:
from aggte.mboot import mboot

In [8]:
inf.shape
# mboot(inf.T, didparams)['se']
ref_se = mboot(inf.T, didparams)
cband_lower = op['att'] - ref_se['crit_val'] * ref_se['se'] 
cband_upper = op['att'] + ref_se['crit_val'] * ref_se['se'] 
sig = (cband_upper < 0) | (cband_lower > 0)
sig[np.isnan(sig)] = False
sig_text = np.where(sig, "*", "")

op['std'] = ref_se['se']
op['l_se'] = cband_lower
op['u_se'] = cband_upper
op['sig'] = sig_text
out_df = pd.DataFrame(op)
name_out_df = ['Group', 'Time', 'ATT(g, t)', 'Post', "Std. Error", "[95% Pointwise", 'Conf. Band]', '']
out_df.columns = name_out_df
out_df

,Group,Time,"ATT(g, t)",Post,Std. Error,[95% Pointwise,Conf. Band],
0,2,2,0.836310,1,0.360671,-0.113341,1.785962,
1,2,3,2.181344,1,0.437505,1.029387,3.333302,*
2,2,4,3.746490,1,0.442707,2.580836,4.912144,*
3,3,2,-0.652337,0,0.366530,-1.617417,0.312743,
4,3,3,1.607831,1,0.341048,0.709846,2.505816,*
5,3,4,3.098372,1,0.425824,1.977171,4.219573,*
6,4,2,-0.107682,0,0.441684,-1.270642,1.055278,
7,4,3,0.251230,0,0.478147,-1.007737,1.510197,
8,4,4,1.439661,1,0.504075,0.112424,2.766899,*


In [12]:
from aggte.aggte import aggte

In [37]:
group = op['group']
att = op['att']
tt = op['year']
V = ref_se['V']
se = ref_se['se']
inffunc = {"inffunc": inf.T}
didpar['bstrap'] = True
didpar['cband'] = True
didpar = didparams
dta = didpar['data']
dta.loc[:, ".w"] = dta['w']
didpar['data'] = dta
didpar['est_method'] = 'dr'

In [38]:
MP = {
  'group': group, 'att': att,
  't': tt, 'DIDparams': didpar,
  'inffunc': inffunc, 'n': dp['n']
}
aggte(MP)



Overall summary of ATT's based on group/cohort aggregation:
   ATT Std. Error [95.0%  Conf. Int.]  
1.9512     0.2753 1.4115       2.4908 *


Group Effects:
   Group  Estimate  Std. Error  [95.0% Simult.   Conf. Band   
0      2    2.2547      0.3705           1.4035      3.1059  *
1      3    2.3531      0.2980           1.6685      3.0377  *
2      4    1.4397      0.5392           0.2009      2.6785  *
---
Signif. codes: `*' confidence band does not cover 0
Control Group:  Never Treated , 
Anticipation Periods:  0
Estimation Method:  Doubly Robust


